# CRUD Operations

## Learning Objectives
- Master Create, Read, Update, Delete operations in SQLAlchemy
- Learn advanced querying techniques and filtering
- Understand bulk operations and batch processing
- Master relationship-based queries and joins
- Learn about query optimization and performance
- Practice real-world data manipulation scenarios

## What You'll Learn
- Basic CRUD operations with SQLAlchemy ORM
- Advanced querying with filters, sorting, and pagination
- Bulk operations for efficient data processing
- Relationship queries and eager loading
- Query optimization techniques
- Error handling and transaction management


## CRUD Operations Overview

CRUD stands for **Create, Read, Update, Delete** - the four basic operations for data manipulation:

- **Create**: Insert new records into the database
- **Read**: Query and retrieve data from the database
- **Update**: Modify existing records
- **Delete**: Remove records from the database

SQLAlchemy provides powerful and intuitive ways to perform these operations using the ORM.


## 1. Setup and Model Definition

Let's start by setting up our environment and defining models for our examples:


In [ ]:
# Setup
from sqlalchemy import create_engine, Column, Integer, String, DateTime, ForeignKey, Text, Boolean, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship
from datetime import datetime

# Create database engine
engine = create_engine('sqlite:///crud_operations.db', echo=True)
Base = declarative_base()
Session = sessionmaker(bind=engine)

print("SQLAlchemy setup complete!")


In [ ]:
# Define models for our CRUD examples
class Author(Base):
    __tablename__ = 'authors'
    
    id = Column(Integer, primary_key=True)
    name = Column(String(100), nullable=False)
    email = Column(String(100), unique=True, nullable=False)
    bio = Column(Text)
    created_at = Column(DateTime, default=datetime.utcnow)
    
    # One-to-many relationship with books
    books = relationship("Book", back_populates="author", cascade="all, delete-orphan")
    
    def __repr__(self):
        return f"<Author(name='{self.name}', email='{self.email}')>"

class Book(Base):
    __tablename__ = 'books'
    
    id = Column(Integer, primary_key=True)
    title = Column(String(200), nullable=False)
    isbn = Column(String(20), unique=True)
    published_year = Column(Integer)
    price = Column(Float)
    is_available = Column(Boolean, default=True)
    created_at = Column(DateTime, default=datetime.utcnow)
    
    # Foreign key to authors table
    author_id = Column(Integer, ForeignKey('authors.id'), nullable=False)
    
    # Many-to-one relationship with author
    author = relationship("Author", back_populates="books")
    
    def __repr__(self):
        return f"<Book(title='{self.title}', author_id={self.author_id})>"

# Create tables
Base.metadata.create_all(engine)

print("Models defined and tables created!")


## 2. CREATE Operations

Let's explore different ways to create records in the database:


In [ ]:
# Create a session
session = Session()

# Method 1: Create single record
author1 = Author(
    name='J.K. Rowling',
    email='jkrowling@example.com',
    bio='Author of the Harry Potter series'
)
session.add(author1)
session.commit()

print(f"Created author: {author1}")

# Method 2: Create multiple records at once
authors_data = [
    Author(name='George Orwell', email='gorwell@example.com', bio='Author of 1984'),
    Author(name='Jane Austen', email='jausten@example.com', bio='Author of Pride and Prejudice'),
    Author(name='Mark Twain', email='mtwain@example.com', bio='Author of Adventures of Huckleberry Finn')
]

session.add_all(authors_data)
session.commit()

print(f"Created {len(authors_data)} authors")

# Method 3: Create with relationship
author = session.query(Author).filter(Author.name == 'J.K. Rowling').first()
book1 = Book(
    title='Harry Potter and the Philosopher\'s Stone',
    isbn='978-0747532699',
    published_year=1997,
    price=12.99,
    author_id=author.id
)
session.add(book1)
session.commit()

print(f"Created book: {book1}")


In [ ]:
# Method 4: Bulk insert for performance
books_data = [
    Book(title='Harry Potter and the Chamber of Secrets', isbn='978-0747538493', published_year=1998, price=13.99, author_id=author.id),
    Book(title='1984', isbn='978-0451524935', published_year=1949, price=10.99, author_id=session.query(Author).filter(Author.name == 'George Orwell').first().id),
    Book(title='Pride and Prejudice', isbn='978-0141439518', published_year=1813, price=9.99, author_id=session.query(Author).filter(Author.name == 'Jane Austen').first().id),
    Book(title='Adventures of Huckleberry Finn', isbn='978-0486280615', published_year=1884, price=11.99, author_id=session.query(Author).filter(Author.name == 'Mark Twain').first().id)
]

session.bulk_save_objects(books_data)
session.commit()

print(f"Bulk inserted {len(books_data)} books")

# Method 5: Using merge for upsert operations
existing_author = Author(
    id=1,  # Assuming author with ID 1 exists
    name='J.K. Rowling Updated',
    email='jkrowling@example.com',
    bio='Updated bio for J.K. Rowling'
)
merged_author = session.merge(existing_author)
session.commit()

print(f"Merged author: {merged_author}")


## 3. READ Operations

Now let's explore various ways to query and retrieve data:


In [ ]:
# Basic queries
print("=== Basic Queries ===")

# Get all authors
all_authors = session.query(Author).all()
print(f"All authors: {len(all_authors)}")
for author in all_authors:
    print(f"  {author}")

# Get all books
all_books = session.query(Book).all()
print(f"\nAll books: {len(all_books)}")
for book in all_books:
    print(f"  {book}")

# Get first author
first_author = session.query(Author).first()
print(f"\nFirst author: {first_author}")

# Get author by ID
author_by_id = session.query(Author).get(1)
print(f"Author with ID 1: {author_by_id}")

# Get one author (raises exception if not found or multiple found)
try:
    one_author = session.query(Author).filter(Author.name == 'J.K. Rowling Updated').one()
    print(f"One author: {one_author}")
except Exception as e:
    print(f"Error: {e}")


In [ ]:
# Advanced filtering
print("\n=== Advanced Filtering ===")

# Filter by single condition
expensive_books = session.query(Book).filter(Book.price > 12.0).all()
print(f"Expensive books (>$12): {len(expensive_books)}")
for book in expensive_books:
    print(f"  {book.title} - ${book.price}")

# Filter by multiple conditions
recent_books = session.query(Book).filter(
    Book.published_year > 1900,
    Book.is_available == True
).all()
print(f"\nRecent available books: {len(recent_books)}")
for book in recent_books:
    print(f"  {book.title} ({book.published_year})")

# Filter with LIKE
harry_books = session.query(Book).filter(Book.title.like('%Harry%')).all()
print(f"\nHarry Potter books: {len(harry_books)}")
for book in harry_books:
    print(f"  {book.title}")

# Filter with IN
specific_years = session.query(Book).filter(Book.published_year.in_([1997, 1998, 1949])).all()
print(f"\nBooks from specific years: {len(specific_years)}")
for book in specific_years:
    print(f"  {book.title} ({book.published_year})")


In [ ]:
# Sorting and ordering
print("\n=== Sorting and Ordering ===")

# Sort by single column
books_by_title = session.query(Book).order_by(Book.title).all()
print("Books sorted by title:")
for book in books_by_title:
    print(f"  {book.title}")

# Sort by multiple columns
books_by_year_title = session.query(Book).order_by(Book.published_year.desc(), Book.title).all()
print("\nBooks sorted by year (desc) then title:")
for book in books_by_year_title:
    print(f"  {book.title} ({book.published_year})")

# Limit and offset (pagination)
print("\n=== Pagination ===")
first_two_books = session.query(Book).order_by(Book.id).limit(2).all()
print("First 2 books:")
for book in first_two_books:
    print(f"  {book.title}")

# Skip first 2, get next 2
next_two_books = session.query(Book).order_by(Book.id).offset(2).limit(2).all()
print("\nNext 2 books:")
for book in next_two_books:
    print(f"  {book.title}")


In [ ]:
# Relationship queries
print("\n=== Relationship Queries ===")

# Get author with their books
author_with_books = session.query(Author).filter(Author.name == 'J.K. Rowling Updated').first()
if author_with_books:
    print(f"Author: {author_with_books.name}")
    print(f"Books by this author: {len(author_with_books.books)}")
    for book in author_with_books.books:
        print(f"  {book.title}")

# Get books with their authors
books_with_authors = session.query(Book).all()
print(f"\nBooks with authors:")
for book in books_with_authors:
    print(f"  {book.title} by {book.author.name}")

# Join queries
print("\n=== Join Queries ===")
books_with_author_info = session.query(Book, Author).join(Author).filter(Book.price > 10.0).all()
print("Expensive books with author info:")
for book, author in books_with_author_info:
    print(f"  {book.title} by {author.name} - ${book.price}")


## 4. UPDATE Operations

Let's explore different ways to update existing records:


In [ ]:
# Method 1: Update by modifying object attributes
print("=== Method 1: Direct Attribute Update ===")

# Find a book to update
book_to_update = session.query(Book).filter(Book.title == 'Harry Potter and the Philosopher\'s Stone').first()
if book_to_update:
    print(f"Before update: {book_to_update.title} - ${book_to_update.price}")
    
    # Update the price
    book_to_update.price = 15.99
    book_to_update.is_available = False
    
    # Commit the changes
    session.commit()
    
    print(f"After update: {book_to_update.title} - ${book_to_update.price} (Available: {book_to_update.is_available})")

# Method 2: Bulk update with update() method
print("\n=== Method 2: Bulk Update ===")

# Update all books published before 1900 to be unavailable
updated_count = session.query(Book).filter(Book.published_year < 1900).update({
    Book.is_available: False
})
session.commit()

print(f"Updated {updated_count} books to unavailable")

# Method 3: Update with conditions
print("\n=== Method 3: Conditional Update ===")

# Update prices for expensive books
expensive_books_updated = session.query(Book).filter(Book.price > 12.0).update({
    Book.price: Book.price * 1.1  # 10% price increase
}, synchronize_session=False)
session.commit()

print(f"Updated prices for {expensive_books_updated} expensive books")


## 5. DELETE Operations

Let's explore different ways to delete records:


In [ ]:
# Method 1: Delete by object
print("=== Method 1: Delete by Object ===")

# Find a book to delete
book_to_delete = session.query(Book).filter(Book.title == 'Adventures of Huckleberry Finn').first()
if book_to_delete:
    print(f"Deleting book: {book_to_delete.title}")
    session.delete(book_to_delete)
    session.commit()
    print("Book deleted successfully")

# Method 2: Bulk delete
print("\n=== Method 2: Bulk Delete ===")

# Delete all unavailable books
deleted_count = session.query(Book).filter(Book.is_available == False).delete()
session.commit()

print(f"Deleted {deleted_count} unavailable books")

# Method 3: Delete with cascade (due to relationship)
print("\n=== Method 3: Cascade Delete ===")

# Create a test author with books
test_author = Author(
    name='Test Author',
    email='test@example.com',
    bio='This author will be deleted'
)
session.add(test_author)
session.commit()

# Add a book to this author
test_book = Book(
    title='Test Book',
    isbn='978-0000000000',
    published_year=2023,
    price=5.99,
    author_id=test_author.id
)
session.add(test_book)
session.commit()

print(f"Created test author with book: {test_author.name}")

# Delete the author (this will cascade delete the book due to cascade="all, delete-orphan")
session.delete(test_author)
session.commit()

print("Test author and associated book deleted (cascade)")

# Verify the book was also deleted
remaining_test_books = session.query(Book).filter(Book.title == 'Test Book').count()
print(f"Remaining test books: {remaining_test_books}")


## 6. Advanced Query Techniques

Let's explore some advanced querying techniques:


In [ ]:
# Aggregation functions
print("=== Aggregation Functions ===")

from sqlalchemy import func

# Count records
total_books = session.query(func.count(Book.id)).scalar()
total_authors = session.query(func.count(Author.id)).scalar()
print(f"Total books: {total_books}")
print(f"Total authors: {total_authors}")

# Average, min, max prices
avg_price = session.query(func.avg(Book.price)).scalar()
min_price = session.query(func.min(Book.price)).scalar()
max_price = session.query(func.max(Book.price)).scalar()
print(f"Average price: ${avg_price:.2f}")
print(f"Min price: ${min_price}")
print(f"Max price: ${max_price}")

# Group by operations
print("\n=== Group By Operations ===")

# Books per author
books_per_author = session.query(
    Author.name,
    func.count(Book.id).label('book_count')
).join(Book).group_by(Author.id, Author.name).all()

print("Books per author:")
for author_name, book_count in books_per_author:
    print(f"  {author_name}: {book_count} books")

# Average price by publication year
avg_price_by_year = session.query(
    Book.published_year,
    func.avg(Book.price).label('avg_price')
).group_by(Book.published_year).order_by(Book.published_year).all()

print("\nAverage price by publication year:")
for year, avg_price in avg_price_by_year:
    print(f"  {year}: ${avg_price:.2f}")


In [ ]:
# Subqueries
print("\n=== Subqueries ===")

# Find authors who have books with price > average price
subquery = session.query(func.avg(Book.price)).scalar()
authors_with_expensive_books = session.query(Author).join(Book).filter(
    Book.price > subquery
).distinct().all()

print(f"Authors with books above average price (${subquery:.2f}):")
for author in authors_with_expensive_books:
    print(f"  {author.name}")

# Exists subquery
print("\n=== Exists Subquery ===")

# Find authors who have at least one book
authors_with_books = session.query(Author).filter(
    session.query(Book).filter(Book.author_id == Author.id).exists()
).all()

print("Authors who have books:")
for author in authors_with_books:
    print(f"  {author.name}")

# Union queries
print("\n=== Union Queries ===")

# Get all unique names from both authors and books (as examples)
author_names = session.query(Author.name.label('name'))
book_titles = session.query(Book.title.label('name'))
all_names = author_names.union(book_titles).all()

print("All unique names (authors and book titles):")
for name_tuple in all_names[:5]:  # Show first 5
    print(f"  {name_tuple.name}")
print("  ...")


## 7. Transaction Management

Let's explore transaction management and error handling:


In [ ]:
# Transaction with rollback
print("=== Transaction with Rollback ===")

try:
    # Start a transaction
    new_author = Author(
        name='Rollback Test Author',
        email='rollback@example.com',
        bio='This will be rolled back'
    )
    session.add(new_author)
    session.flush()  # Flush to get the ID
    
    # Add a book
    new_book = Book(
        title='Rollback Test Book',
        isbn='978-1111111111',
        published_year=2023,
        price=9.99,
        author_id=new_author.id
    )
    session.add(new_book)
    
    # Intentionally cause an error (duplicate email)
    duplicate_author = Author(
        name='Duplicate Author',
        email='rollback@example.com',  # Same email - will cause error
        bio='This will cause an error'
    )
    session.add(duplicate_author)
    session.commit()  # This will fail
    
except Exception as e:
    print(f"Error occurred: {e}")
    session.rollback()
    print("Transaction rolled back successfully")

# Verify rollback worked
rollback_author = session.query(Author).filter(Author.name == 'Rollback Test Author').first()
rollback_book = session.query(Book).filter(Book.title == 'Rollback Test Book').first()
print(f"Rollback author exists: {rollback_author is not None}")
print(f"Rollback book exists: {rollback_book is not None}")

# Context manager for transactions
print("\n=== Context Manager Transaction ===")

from contextlib import contextmanager

@contextmanager
def transaction(session):
    try:
        yield session
        session.commit()
    except Exception as e:
        session.rollback()
        raise e

# Use the context manager
try:
    with transaction(session):
        author = Author(
            name='Context Manager Author',
            email='context@example.com',
            bio='Created with context manager'
        )
        session.add(author)
        # This will succeed
        print("Author created successfully with context manager")
except Exception as e:
    print(f"Error in context manager: {e}")

# Close the session
session.close()

print("\n" + "="*50)
print("CRUD Operations examples completed successfully!")
print("="*50)


## Summary

You've learned about:

1. **CREATE Operations** - Single records, bulk inserts, merge operations
2. **READ Operations** - Basic queries, filtering, sorting, pagination, relationships
3. **UPDATE Operations** - Direct updates, bulk updates, conditional updates
4. **DELETE Operations** - Single deletes, bulk deletes, cascade deletes
5. **Advanced Queries** - Aggregations, subqueries, joins, unions
6. **Transaction Management** - Rollbacks, context managers, error handling

## Key Takeaways

- **CRUD Operations**: Master the four basic database operations
- **Query Optimization**: Use appropriate methods for different scenarios
- **Bulk Operations**: Use `bulk_save_objects()` and `update()` for performance
- **Relationships**: Leverage SQLAlchemy relationships for complex queries
- **Transactions**: Always handle errors and use proper transaction management
- **Performance**: Choose the right method for your use case (single vs bulk operations)

## Best Practices

- Always use transactions for data integrity
- Use bulk operations for large datasets
- Leverage relationships for complex queries
- Handle errors gracefully with try/except blocks
- Use context managers for automatic transaction management
- Close sessions properly to free resources

Ready to practice? Move on to the exercise notebook!
e an